In [2]:
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm #to check  progress bar
import numpy as np

In [3]:
path = '../Data'
full_path_list = [ path + '/' + f for \
                 f in os.listdir(path) if os.path.isfile(os.path.join(path,f)) ]
full_path_list

['../Data/dailyActivity_merged.csv',
 '../Data/dailyCalories_merged.csv',
 '../Data/dailyIntensities_merged.csv',
 '../Data/dailySteps_merged.csv',
 '../Data/sleepDay_merged.csv',
 '../Data/weightLogInfo_merged.csv']

In [4]:
#The fitbit dataset is inclusive of hours and minutes datasets as well. However, for the purpose of this exercise,
# it is chosen to analyze daily aggregated data only.

# connect to sql db
con = sql.connect("fitbit.db")
cur = con.cursor()

def get_table_name(full_path_list, i):
    return full_path_list[i].split("/")[-1].split('.')[0]

pbar = tqdm(total=len(full_path_list), desc='[Loading Data...]')
for i in range(0,len(full_path_list)):
    pd.read_csv(full_path_list[i]).to_sql(get_table_name(full_path_list, i), con, if_exists='append', index=False)
    pbar.update(1)
pbar.close()

# simple sql query test
df = pd.read_sql(f'SELECT * FROM {get_table_name(full_path_list, 0)}', con)

df.head()

[Loading Data...]: 100%|█████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.78it/s]


,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,2016-04-12,13162,8.50,8.50,0.0,1.88,0.55,6.06,0.0,25,13,328,728,1985
1,1503960366,2016-04-13,10735,6.97,6.97,0.0,1.57,0.69,4.71,0.0,21,19,217,776,1797
2,1503960366,2016-04-14,10460,6.74,6.74,0.0,2.44,0.40,3.91,0.0,30,11,181,1218,1776
3,1503960366,2016-04-15,9762,6.28,6.28,0.0,2.14,1.26,2.83,0.0,29,34,209,726,1745
4,1503960366,2016-04-16,12669,8.16,8.16,0.0,2.71,0.41,5.04,0.0,36,10,221,773,1863


In [5]:
# list all tables
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()
print(f'Total of {tables} tables in database.')

Total of [('dailyActivity_merged',), ('dailyCalories_merged',), ('dailyIntensities_merged',), ('dailySteps_merged',), ('sleepDay_merged',), ('weightLogInfo_merged',)] tables in database.


In [5]:
# check for sample data and data size
dailyActivity_df = pd.read_sql(f'SELECT * FROM dailyActivity_merged', con)

print(len(dailyActivity_df))

dailyActivity_df.head()

940


,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,4/12/2016,13162,8.50,8.50,0.0,1.88,0.55,6.06,0.0,25,13,328,728,1985
1,1503960366,4/13/2016,10735,6.97,6.97,0.0,1.57,0.69,4.71,0.0,21,19,217,776,1797
2,1503960366,4/14/2016,10460,6.74,6.74,0.0,2.44,0.40,3.91,0.0,30,11,181,1218,1776
3,1503960366,4/15/2016,9762,6.28,6.28,0.0,2.14,1.26,2.83,0.0,29,34,209,726,1745
4,1503960366,4/16/2016,12669,8.16,8.16,0.0,2.71,0.41,5.04,0.0,36,10,221,773,1863


In [6]:
# update date to match sqlite format
update_date = """
UPDATE dailyActivity_merged set ActivityDate =
    SUBSTR(ActivityDate, -4)
    || "-" ||
    CASE
        WHEN LENGTH(
            SUBSTR( -- picking month info
                ActivityDate, 1, INSTR(ActivityDate, '/') - 1
            )
        ) > 1 THEN
            SUBSTR( -- picking month info
                ActivityDate, 1, INSTR(ActivityDate, '/') - 1
            )
        ELSE '0' ||
            SUBSTR( -- picking month info
                ActivityDate, 1, INSTR(ActivityDate, '/') - 1
            )
    END
    || "-" ||
    CASE
    WHEN LENGTH(
        SUBSTR( -- picking day info
            SUBSTR(ActivityDate, INSTR(ActivityDate, '/') + 1), -- pick substring starting after first /
            1,  -- start new substring at first character of newly selected substring
            INSTR(SUBSTR(ActivityDate, INSTR(ActivityDate, '/') + 1), '/') - 1 -- go all the way to next /
        )
    ) > 1 THEN
        SUBSTR( -- picking day info
            SUBSTR(ActivityDate, INSTR(ActivityDate, '/') + 1), -- pick substring starting after first /
            1,  -- start new substring at first character of newly selected substring
            INSTR(SUBSTR(ActivityDate, INSTR(ActivityDate, '/') + 1), '/') - 1 -- go all the way to next /
        )
    ELSE '0' ||
        SUBSTR( -- picking day info
            SUBSTR(ActivityDate, INSTR(ActivityDate, '/') + 1), -- pick substring starting after first /
            1,  -- start new substring at first character of newly selected substring
            INSTR(SUBSTR(ActivityDate, INSTR(ActivityDate, '/') + 1), '/') - 1 -- go all the way to next /
        )
    END;
"""
cur.execute(update_date)
con.commit()

In [7]:
# update data to match sql lite format
update_date = """
UPDATE sleepDay_merged set SleepDay =
    SUBSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), -4)
    || "-" ||
    CASE
        WHEN LENGTH(
            SUBSTR( -- picking month info
                SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), 1, INSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), '/') - 1
            )
        ) > 1 THEN
            SUBSTR( -- picking month info
                SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), 1, INSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), '/') - 1
            )
        ELSE '0' ||
            SUBSTR( -- picking month info
                SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), 1, INSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), '/') - 1
            )
    END
    || "-" ||
    CASE
    WHEN LENGTH(
        SUBSTR( -- picking day info
            SUBSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), INSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), '/') + 1), -- pick substring starting after first /
            1,  -- start new substring at first character of newly selected substring
            INSTR(SUBSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), INSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), '/') + 1), '/') - 1 -- go all the way to next /
        )
    ) > 1 THEN
        SUBSTR( -- picking day info
            SUBSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), INSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), '/') + 1), -- pick substring starting after first /
            1,  -- start new substring at first character of newly selected substring
            INSTR(SUBSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), INSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), '/') + 1), '/') - 1 -- go all the way to next /
        )
    ELSE '0' ||
        SUBSTR( -- picking day info
            SUBSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), INSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), '/') + 1), -- pick substring starting after first /
            1,  -- start new substring at first character of newly selected substring
            INSTR(SUBSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), INSTR(SUBSTR(SleepDay, 1, LENGTH(SleepDay) - 12), '/') + 1), '/') - 1 -- go all the way to next /
        )
    END;
"""
cur.execute(update_date)
con.commit()

In [8]:
# extract day,month,year
full_info_activity = """
SELECT *,
    STRFTIME('%d',ActivityDate) day,
    STRFTIME('%m',ActivityDate) month,
    STRFTIME('%Y',ActivityDate) year,
    STRFTIME('%w',ActivityDate) dow
FROM dailyActivity_merged;
"""

full_dailyActivity_df = pd.read_sql(full_info_activity, con)

In [9]:
full_dailyActivity_df.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,day,month,year,dow
0,1503960366,2016-04-12,13162,8.50,8.50,0.0,1.88,0.55,6.06,0.0,25,13,328,728,1985,12,04,2016,2
1,1503960366,2016-04-13,10735,6.97,6.97,0.0,1.57,0.69,4.71,0.0,21,19,217,776,1797,13,04,2016,3
2,1503960366,2016-04-14,10460,6.74,6.74,0.0,2.44,0.40,3.91,0.0,30,11,181,1218,1776,14,04,2016,4
3,1503960366,2016-04-15,9762,6.28,6.28,0.0,2.14,1.26,2.83,0.0,29,34,209,726,1745,15,04,2016,5
4,1503960366,2016-04-16,12669,8.16,8.16,0.0,2.71,0.41,5.04,0.0,36,10,221,773,1863,16,04,2016,6


### 1. Count the number of users present in dailyActivity_merged table.

In [13]:
pd.read_sql('SELECT COUNT(DISTINCT Id) FROM dailyActivity_merged', con)

,COUNT(DISTINCT Id)
0,33


### 2. Compute average calories, steps, and distance by Id as day of week.

In [16]:
query = """
SELECT 
  Id,
  STRFTIME('%w', ActivityDate) dow,
  ROUND(AVG(Calories),2) AS avg_calories,
  ROUND(AVG(TotalSteps),2) AS avg_steps,
  ROUND(AVG(TotalDistance),2) AS avg_distance
FROM dailyActivity_merged
GROUP BY Id, STRFTIME('%w', ActivityDate);
"""
pd.read_sql(query, con)

,Id,dow,avg_calories,avg_steps,avg_distance
0,1503960366,0,1769.00,10101.50,6.57
1,1503960366,1,1939.25,13780.75,8.96
2,1503960366,2,1967.80,13946.60,8.92
3,1503960366,3,1868.80,12656.60,8.23
4,1503960366,4,1481.60,9500.60,6.10
...,...,...,...,...,...
223,8877689391,2,3221.20,14924.60,12.52
224,8877689391,3,3672.40,19705.40,16.84
225,8877689391,4,3082.60,15503.00,12.94
226,8877689391,5,3518.50,15785.00,12.55


### 3. Create a new dataframe by combining daily activity data and sleep data.

In [23]:
#  Create a new dataframe by combining daily activity data and sleep data to dailyActivity_merged.
join_query = """
SELECT 
  A.Id,
  A.ActivityDate,
  A.SedentaryMinutes,
  A.LightlyActiveMinutes,
  S.TotalMinutesAsleep
FROM 
  dailyActivity_merged A
INNER JOIN sleepDay_merged S
ON 
  A.Id = S.Id AND
    A.ActivityDate = S.SleepDay;
"""
activity_sleep_df = pd.read_sql(join_query, con)

activity_sleep_df

,Id,ActivityDate,SedentaryMinutes,LightlyActiveMinutes,TotalMinutesAsleep
0,1503960366,2016-04-12,728,328,327
1,1503960366,2016-04-13,776,217,384
2,1503960366,2016-04-15,726,209,412
3,1503960366,2016-04-16,773,221,340
4,1503960366,2016-04-17,539,164,700
...,...,...,...,...,...
408,8792009665,2016-04-30,749,301,343
409,8792009665,2016-05-01,834,79,503
410,8792009665,2016-05-02,916,101,415
411,8792009665,2016-05-03,739,156,516


### 4. Fit a regression line between total steps, total distance and calories to report the intercept and slope. Also, compare the regression coefficients.

In [36]:
from sklearn.linear_model import LinearRegression
X = full_dailyActivity_df[["TotalSteps","TotalDistance"]].values
y = full_dailyActivity_df["Calories"].values
lr = LinearRegression()
lr.fit(X,y)

print("Intercept -",lr.intercept_)
print("Coefficient -",lr.coef_)

Intercept - 1789.4707308482525
Coefficient - [-2.13629587e-01  3.90881416e+02]


### 5. Write your observation on shape of the dataframe and report its descriptive statistics for Q's 2 and 3.

In [40]:
print(activity_sleep_df.shape)
activity_sleep_df.describe()

(413, 5)


,Id,SedentaryMinutes,LightlyActiveMinutes,TotalMinutesAsleep
count,4.130000e+02,413.000000,413.000000,413.000000
mean,5.000979e+09,712.169492,216.854722,419.467312
std,2.060360e+09,165.957432,87.164687,118.344679
min,1.503960e+09,0.000000,2.000000,58.000000
25%,3.977334e+09,631.000000,158.000000,361.000000
50%,4.702922e+09,717.000000,208.000000,433.000000
75%,6.962181e+09,783.000000,263.000000,490.000000
max,8.792010e+09,1265.000000,518.000000,796.000000
